In [1]:
!pip3 install -q -U git+https://github.com/huggingface/transformers.git

!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip3 install -q -U git+https://github.com/huggingface/optimum.git

!pip3 install -q -U flash-attn
!pip install -q -U bitsandbytes
!pip install -q -U accelerate


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
import torch
import re
import json

c:\Users\sofik\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [13]:
model_id = "TheBloke/CodeLlama-7B-Instruct-GPTQ"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
          bnb_4bit_use_double_quant=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.bfloat16
 )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.float16,
    revision = "main",
    trust_remote_code = True,
    cache_dir = ''
    )
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [34]:
# Use prompts created for the zero-shot task
with open("/content/zero_prompts.json", "r") as file:
    zero_prompts = json.load(file)


In [22]:
# Delimiters
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

system_prompt="""You are a helpful assistant. You provide accurate information."""

# Define a stream *without* function calling capabilities
def stream(user_prompt):
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").input_ids.cuda()
    streamer = TextStreamer(tokenizer)

    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
        generated_text = model.generate(inputs=inputs, streamer=streamer, max_new_tokens=500, temperature=0.01, do_sample=True)

    # Decode the generated text
    generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

    return generated_text




In [40]:
cllama_ans = {}
# for question in zero_prompts:
#   answer= stream(zero_prompts[question])
#   answer_clean= re.findall(r'\{([^{}]*)\}', answer)
#   cllama_ans[question] = answer_clean
#   torch.cuda.empty_cache()

# two runs
for i in range(2):
    for question in zero_prompts:
        answer = stream(zero_prompts[question])
        answer_clean = re.findall(r'\{([^{}]*)\}', answer) # extract dict from answer
        answer_dict = [eval(i) for i in answer_clean]

        if question not in cllama_ans:
            cllama_ans[question] = []
        cllama_ans[question].append(answer_dict)

        torch.cuda.empty_cache()


<s> [INST] <<SYS>>
You are a helpful assistant. You provide accurate information.
<</SYS>>

In the following you will be given a question followed several possible answer categories. Please     estimate how a representative sample of 7900 US-Americans would have answered the question in 2020 by providing the    number of respondents for each answer option in percent (output a dictionary object with answer category number as    key and answer frequency in percent as value. Only provide the number without the percentage sign).    Here is the questions: There has been some discussion about abortion during recent
years. Which one of the opinions on this page best agrees with
your view?
1. By law, abortion should never be permitted
2. The law should permit abortion only in case of rape, incest, or
when the woman's life is in danger
3. The law should permit abortion other than for
rape/incest/danger to woman but only after need clearly
established
4. By law, a woman should always be able to 

In [42]:
cllama_ans

cllama_answers = json.dumps(cllama_ans)
with open("cllama_answers.json", "w") as file:
    file.write(cllama_answers)

In [50]:
import json

with open("cllama_answers.json", "r") as file:
    cllama_answers = json.load(file)

In [19]:
answers

{'V201336': [['1: 10.5%, 2: 20.5%, 3: 48.5%, 4: 28.5%'],
  ['\n1: 10.3%,\n2: 21.5%,\n3: 32.8%,\n4: 26.9%\n']],
 'V201342x': [['\n1: 30.4%\n2: 21.2%\n3: 12.3%\n4: 36.3%\n5: 10.1%\n6: 4.7%\n7: 0.9%\n'],
  ['\n1: 30.5%\n2: 21.5%\n3: 12.5%\n4: 26.5%\n5: 10.5%\n6: 5.5%\n7: 0.5%\n']],
 'V202332': [['\n1: 10.5%,\n2: 22.5%,\n3: 32.5%,\n4: 15.5%,\n5: 10.5%\n'],
  ['\n1: 10.5%,\n2: 22.5%,\n3: 32.5%,\n4: 15.5%,\n5: 10.5%\n']],
 'V202333': [['\n1: 20.5%,\n2: 22.5%,\n3: 31.5%,\n4: 15.5%,\n5: 10.5%\n'],
  ['\n1: 20.5%,\n2: 22.5%,\n3: 31.5%,\n4: 15.5%,\n5: 10.5%\n']],
 'V202337': [['\n1: 30.5%,\n2: 27.5%,\n3: 31.5%\n'],
  ['\n1: 30.5%,\n2: 27.5%,\n3: 31.5%\n']],
 'V202341x': [['\n"Favor a great deal": 10.5%,\n"Favor a moderate amount": 22.5%,\n"Favor a little": 25.5%,\n"Neither favor nor oppose": 42.5%,\n"Oppose a little": 15.5%,\n"Oppose a moderate amount": 17.5%,\n"Oppose a great deal": 10.5%\n'],
  ['\n"Favor a great deal": 10.5%,\n"Favor a moderate amount": 22.5%,\n"Favor a little": 25.5%,\n"Neit

In [25]:
for question, answers in cllama_answers.items():
    processed_answers = []
    for answer in answers:
        print(answer)

['1: 10.5%, 2: 20.5%, 3: 48.5%, 4: 28.5%']
['\n1: 10.3%,\n2: 21.5%,\n3: 32.8%,\n4: 26.9%\n']
['\n1: 30.4%\n2: 21.2%\n3: 12.3%\n4: 36.3%\n5: 10.1%\n6: 4.7%\n7: 0.9%\n']
['\n1: 30.5%\n2: 21.5%\n3: 12.5%\n4: 26.5%\n5: 10.5%\n6: 5.5%\n7: 0.5%\n']
['\n1: 10.5%,\n2: 22.5%,\n3: 32.5%,\n4: 15.5%,\n5: 10.5%\n']
['\n1: 10.5%,\n2: 22.5%,\n3: 32.5%,\n4: 15.5%,\n5: 10.5%\n']
['\n1: 20.5%,\n2: 22.5%,\n3: 31.5%,\n4: 15.5%,\n5: 10.5%\n']
['\n1: 20.5%,\n2: 22.5%,\n3: 31.5%,\n4: 15.5%,\n5: 10.5%\n']
['\n1: 30.5%,\n2: 27.5%,\n3: 31.5%\n']
['\n1: 30.5%,\n2: 27.5%,\n3: 31.5%\n']
['\n"Favor a great deal": 10.5%,\n"Favor a moderate amount": 22.5%,\n"Favor a little": 25.5%,\n"Neither favor nor oppose": 42.5%,\n"Oppose a little": 15.5%,\n"Oppose a moderate amount": 17.5%,\n"Oppose a great deal": 10.5%\n']
['\n"Favor a great deal": 10.5%,\n"Favor a moderate amount": 22.5%,\n"Favor a little": 25.5%,\n"Neither favor nor oppose": 42.5%,\n"Oppose a little": 15.5%,\n"Oppose a moderate amount": 17.5%,\n"Oppose a grea

In [57]:
import re

map_answers = {
    "Favor a great deal": 1,
    "Favor a moderate amount": 2,
    "Favor a little": 3,
    "Neither favor nor oppose": 4,
    "Oppose a little": 5,
    "Oppose a moderate amount": 6,
    "Oppose a great deal": 7
}

processed_cllama_ans = {}

for question, answers in cllama_answers.items():
    processed_answers = []
    for answer in answers:
        clean_answer = re.sub(r'"[^"]*"', '', answer[0])
        clean_answer = clean_answer.replace('%', ', ')
        # Map textual answer to its corresponding numeric value
        # if clean_answer.strip() in map_answers:
        #     clean_answer = map_answers[clean_answer.strip()]

        # Replace '\n' with '' if it's at the beginning or following a comma
        clean_answer = re.sub(r'(?:^|,)\s*\n', '', clean_answer)
        # Map textual answer to its corresponding numeric value

        processed_answers.append(clean_answer)
    processed_cllama_ans[question] = processed_answers


In [64]:
processed_cllama_ans
# ended up editing them manually

with open('cllama_ans_clean.txt', 'r') as file:
    # Read the content of the file
    cllama_ans = file.readlines()

cllama_ans = [line.strip() for line in cllama_ans]

In [69]:
cllama_ans

["{'V201336': ['1: 10.5, , 2: 20.5, , 3: 48.5, , 4: 28.5', '1: 10.3, 2: 21.5, 3: 32.8, 4: 26.9'],",
 "'V201342x': ['1: 30.4, 2: 21.2, 3: 12.3, 4: 36.3, 5: 10.1, 6: 4.7, 7: 0.9', '1: 30.5, 2: 21.5, 3: 12.5, 4: 26.5, 5: 10.5, 6: 5.5, 7: 0.5'],",
 "'V202332': ['1: 10.5, 2: 22.5, 3: 32.5, 4: 15.5, 5: 10.5','1: 10.5, 2: 22.5, 3: 32.5, 4: 15.5, 5: 10.5'],",
 "'V202333': ['1: 20.5, 2: 22.5, 3: 31.5, 4: 15.5, 5: 10.5','1: 20.5, 2: 22.5, 3: 31.5, 4: 15.5, 5: 10.5'],",
 "'V202337': ['1: 30.5, 2: 27.5, 3: 31.5', '1: 30.5, 2: 27.5, 3: 31.5'],",
 "'V202341x': ['1: 10.5, 2: 22.5, 3: 25.5, 4: 42.5, 5: 15.5, 6: 17.5, 7: 10.5','1: 10.5, 2: 22.5, 3: 25.5, 4: 42.5, 5: 15.5, 6: 17.5, 7: 10.5'],",
 "'V202344x': ['1: 10.5, 2: 22.5, 3: 25.5, 4: 42.5, 5: 15.5, 6: 17.5, 7: 10.5','1: 10.5, 2: 22.5, 3: 25.5, 4: 42.5, 5: 15.5, 6: 17.5, 7: 10.5'],",
 "'V202232': ['1: 30.5, 2: 22.5, 3: 25.5, 4: 10.5, 5: 5.5','1: 30.5, 2: 21.5, 3: 27.5, 4: 10.5, 5: 5.5'],",
 "'V202233': ['1: 30.5, 2: 27.5, 3: 21.5, 4: 18.5', '1: 30.